# Optimization exercise

## Goal: Train the 2nd order polynomial predictor using both gradient descent and stochastic gradient descent. Optimize the stepsizes and compare against scikit-learn implementation

1. Download data from https://drive.google.com/file/d/0Bz9_0VdXvv9bUUNlUTVrMF9VcVU/view?usp=sharing.
2. Create a function psi(x), which transforms features AST (assists), REB (rebounds) and STL (steals) into 2nd order polynomial features (add each feature squared and each pair of features multiplied with every other)
3. Create a transformed data matrix X, where each x is mapped to psi(x).
4. Create a function p2(x,w), which outputs the value of the polynomial at x for given parameters w.
5. Create a function Loss(X,y,w), which computes the squared loss of predicting y from X by p2(x,w) using parameters w. Take variable PTS as y. We will predict scored points based on assists, rebounds and steals.
6. Code up the gradient descent. It should input a point w and a stepsize.
7. Choose an arbitrary point and stepsize. Run gradient descent for 100 iterations and compute the Loss after each iteration. How does the loss behave? Does it converge to something?
8. Can you find the stepsize, for which the loss is smallest after 100 iterations?

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import random
import pandas as pd
import copy
from ipykernel import kernelapp as app

In [2]:
nb = pd.read_csv(r"/Users/Shrey/Downloads/nba_games_2013_2015 (1).csv", delimiter=';')
x = nb[['AST','REB','STL']].values
y = nb[['PTS']].values

In [3]:
def psi(q):
    f = [q[0]**2 , q[1]**2 , q[2]**2 , q[0]*q[1] , q[0]*q[2] , q[1]*q[2] ]
    return f

In [4]:
psi(x[0])

[1681, 1849, 196, 1763, 574, 602]

In [5]:
X = []
for i in x:
     X.append(psi(i))

In [6]:
X = np.array(X)
X.shape
X

array([[1681, 1849,  196, 1763,  574,  602],
       [ 529, 1849,   64,  989,  184,  344],
       [ 400, 1521,   49,  780,  140,  273],
       ...,
       [ 529, 2704,   64, 1196,  184,  416],
       [ 529, 1681,  121,  943,  253,  451],
       [ 289, 1936,   16,  748,   68,  176]])

In [7]:
k = np.array([[1,1,1,1,1,1]]).transpose()
k.shape

(6, 1)

In [8]:
def p2(x,w):
    # Take dot product of x values
    return np.matmul(x,w)

In [9]:
def loss(X,y,w):
    total_loss = []
    for i in X:
        i = np.array([i])
        loss = (y - p2(i,w))**2
        total_loss.append(loss)
    return sum(total_loss)

## Gradient Descent implementation

Loss Function = $\displaystyle\frac{1}{2m}||Ax-b||_{2}^2$ or simply $J(\theta) = 1/2m \sum_{i=1}^{m} (h(\theta)^{(i)} - y^{(i)})^2$

Gradient = $\displaystyle\frac{1}{m}A^T(Ax-b)$ or simply $\frac{\partial J(\theta)}{\partial \theta_j} = 1/m\sum_{i=1}^{m}(h(\theta^{(i)} - y^{(i)}).X_j^{(i)}$


In [1]:
# Our loss function not convex won't converge

In [17]:
def gradient_descent(y,X,learning_rate,iterations,theta):
    #theta is initial start
    past_thetas = []
    past_costs = []
    grad = []
    for i in range(0,iterations):
        #compute initial prediction
        prediction = X.dot(theta)
        #compute error 
        error = np.sum((y - prediction)**2)/(2*len(y))
        #add error
        past_costs.append(error)
        #update parameters using gradient descent -> for least squares formula is X^t(Xw-Y)
        gradient = (X.transpose().dot((X.dot(theta) - y)))/len(y)
        grad.append(gradient)
        theta = theta - learning_rate*gradient
        past_thetas.append(theta)
        
        
    return past_thetas, past_costs , grad

In [18]:
past_thetas, past_costs , grad = gradient_descent(y,X,0.0001,10,k)

In [19]:
len(past_costs)

10

## Implementing Stochastic Gradient Descent 

In [20]:
def stochastic_gradient_descent(y,X,learning_rate,iterations,theta):
    past_thetas = []
    past_costs = []
    grad = []
    m = len(y)
    for j in range(0,iterations):
        cost = 0
        for i in range(0,m):
            rand = np.random.randint(0,m)
            X_i = X[rand,:].reshape(1,X.shape[1])
            y_i = y[rand].reshape(1,1)
            prediction = np.dot(X_i,theta)
            gradient = (X_i.T.dot((prediction - y_i)))/m
            grad.append(gradient)
            theta = theta -learning_rate*gradient
            past_thetas.append(theta)
            error = np.sum((y_i - prediction)**2)/(2*m)
            past_costs.append(error)
            
    return past_thetas , past_costs , grad   

In [21]:
past_thetas , past_costs , grad = stochastic_gradient_descent(y,X,0.01,10,k)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: RuntimeWarning: overflow encountered in square
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in subtract
  from ipykernel import kernelapp as app


In [27]:
X_i = X[7,:].reshape(1,X.shape[1])
y_i = y[7].reshape(1,1)
y_i

array([[109]])

In [22]:
len(past_costs)

73800

## Analytical Solution

In [15]:
len(y)

7380

Since our Loss Function is equal to $\displaystyle\frac{1}{2m}||Ax-b||_{2}^2$ we need to find a x that minimizes the error. If we're lucky then our unique x will be equal to $(X^TX)^{-1}X^Ty$ and this is only possible if  $(X^TX)$ is invertible. Gradient descent doesn't always mean we will reach our global optimum so if we our data isn't too large we can solve this directly 


In [86]:
## Unique solution
unique = np.linalg.inv(X.transpose().dot(X)).dot(X.transpose().dot(y))
unique

array([[ 0.05828381],
       [ 0.0230394 ],
       [ 0.13298273],
       [ 0.01246142],
       [-0.03234214],
       [ 0.02218657]])